# [한글 감성 분석 실습: 한글 전처리 및 TDM & TFIDF]

### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- y: Code로 변경
- m: Markdown으로 변경

## 1. 모듈 불러오기

#### import '불러올 패키지명' as '그 패키지를 파이썬에서 사용할 이름'

In [1]:
import konlpy
from konlpy.tag import Hannanum, Kkma, Komoran

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk

import re
import sys
import os
import time
import random
import pandas as pd
import numpy as np

## 2. 데이터 불러오기: English Movie Review Data

#### 데이터 구조
- 데이터: 한글 영화 리뷰 데이터  
- 관측치 건수: 13697건 (train: 12997 / test: 700)
- 변수 개수: 설명변수 1개 / 반응변수 1개

#### 설명 변수(원인: 예측값을 설명할 수 있는 변수)      
- review: 영화 리뷰 데이터(Document)


#### 반응 변수(결과: 예측하고자 하는 값)
- sentiment: 리뷰의 감정 (긍정:1/ 부정:0)

In [2]:
train_data = pd.read_csv('data/korean_train.csv',encoding='utf-8', engine='python', index_col=0)
test_data = pd.read_csv('data/korean_test.csv', encoding='utf-8', engine='python',index_col=0)

## 3. 데이터 구성

### - 3.1 train 데이터 및 test 데이터 갯수 확인

In [3]:
print('DATA COLUMNS NAMES: {}'.format(list(train_data.columns)))
print('\n')
print('TRAIN DATA SHAPE: {}'.format(train_data.shape))
print('TEST  DATA SHAPE: {}'.format(test_data.shape))

DATA COLUMNS NAMES: ['reviews', 'sentiment']


TRAIN DATA SHAPE: (12997, 2)
TEST  DATA SHAPE: (700, 2)


### - 3.2 train/test 데이터 병합

In [4]:
whole_data = pd.concat([train_data, test_data]).reset_index(drop=True)

In [5]:
whole_data.head() #.head(): 상위 5개 예시

,reviews,sentiment
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


## 4. 데이터 텍스트 전처리

### - 4.1 데이터 기호 및 문법 정규화

https://python.bakyeono.net/chapter-11-2.html

#### re모듈은 패턴과 매치하는 텍스트를 찾고 조각하는 기능 제공

- re.compile(pattern)            : 패턴 문자열 pattern을 패턴 객체로 컴파일한다.   
- re.search(pattern, string)     : string에서 pattern과 매치하는 텍스트를 탐색한다. (임의 지점 매치)
- re.match(pattern, string)      : string에서 pattern과 매치하는 텍스트를 탐색한다. (시작점 매치)
- re.fullmatch(pattern, string)  : string에서 pattern과 매치하는 텍스트를 탐색한다. (전체 매치)
- re.sub(pattern, replacd, string)  : string에서 pattern과 매치하는 텍스트를 repl로 치환한다.
- re.split(pattern, string)	     : string을 pattern을 기준으로 나눈다.


In [6]:
# re.sub(pattern, replacd, string) example
re.sub("-","0","991225-1234567")

'99122501234567'

In [7]:
docs_tr = [re.sub(r"\%\$ ?\([^)]+\)-", "", x) for x in whole_data.reviews.values.tolist()]
docs_tr = [re.sub("\'", " ", x) for x in docs_tr]
docs_tr = [re.sub('\S*@\S*\s?', '', x) for x in docs_tr]
docs_tr = [re.sub('\s+', ' ', x) for x in docs_tr]
docs_tr = [re.sub('[^\w\s]','',x) for x in docs_tr]

In [8]:
# Original data
pd.DataFrame(whole_data.reviews.head())

,reviews
0,아 더빙.. 진짜 짜증나네요 목소리
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...


In [9]:
# Preprocessed data (Normalization)
pd.DataFrame(docs_tr, columns = ['preprocessed_reviews']).head()

,preprocessed_reviews
0,아 더빙 진짜 짜증나네요 목소리
1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다
3,교도소 이야기구먼 솔직히 재미는 없다평점 조정
4,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...


In [10]:
whole_data.reviews = docs_tr

In [11]:
whole_data.head()

,reviews,sentiment
0,아 더빙 진짜 짜증나네요 목소리,0
1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


## 5. 토큰화 전처리

### - 5.1 토큰화

#### - Hannanum: 한나눔. KAIST Semantic Web Research Center 개발.
#### http://semanticweb.kaist.ac.kr/hannanum/
#### - Kkma: 꼬꼬마. 서울대학교 IDS(Intelligent Data Systems) 연구실 개발.
#### http://kkma.snu.ac.kr/
#### - Komoran: 코모란. Shineware에서 개발.
#### https://github.com/shin285/KOMORAN
#### - Mecab: 메카브. 일본어용 형태소 분석기를 한국어를 사용할 수 있도록 수정.
#### https://bitbucket.org/eunjeon/mecab-ko
#### - Open Korean Text: 오픈 소스 한국어 분석기. 과거 트위터 형태소 분석기.
#### https://github.com/open-korean-text/open-korean-text

In [12]:
# hannanum = Hannanum()
# kkma = Kkma()
# komoran = Komoran()

# nouns: 명사 추출
# morphs: 형태소 추출
# pos: 품사 부착

hannanum = Hannanum()
docs_tr = [hannanum.morphs(review) for review in whole_data.reviews.values.tolist()]

In [13]:
# 토큰화 결과
whole_data.reviews = docs_tr

In [14]:
whole_data.head()

,reviews,sentiment
0,"[아, 더빙, 진짜, 짜증나, 네, 요, 목소리]",0
1,"[흠포스터보, 이, 고, 초딩영화줄오버연기조차, 가볍, 지, 않, 구나]",1
2,[너무재밓었다그래서보는것을추천한다],0
3,"[교도소, 이야기구먼, 솔직히, 재미, 는, 없다평점, 조정]",0
4,"[사이몬페그, 의, 익살, 스런, 연기, 가, 돋보이, 었던, 영화스파이더맨, 에서...",1


## 6. 토큰화 데이터 탐색

### - 6.1 전체 문서에 대한 토큰 생성

In [15]:
tokens = [tok for sublist in docs_tr for tok in sublist]

In [16]:
tokens

['아',
 '더빙',
 '진짜',
 '짜증나',
 '네',
 '요',
 '목소리',
 '흠포스터보',
 '이',
 '고',
 '초딩영화줄오버연기조차',
 '가볍',
 '지',
 '않',
 '구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소',
 '이야기구먼',
 '솔직히',
 '재미',
 '는',
 '없다평점',
 '조정',
 '사이몬페그',
 '의',
 '익살',
 '스런',
 '연기',
 '가',
 '돋보이',
 '었던',
 '영화스파이더맨',
 '에서',
 '늙',
 '어',
 '보이',
 '기',
 '만',
 '하',
 '었던',
 '커스틴',
 '던스트',
 '가',
 '너무나',
 '도',
 '이쁘',
 '어',
 '보이',
 '었다',
 '막',
 '걸음마',
 '떼',
 'ㄴ',
 '3세',
 '부터',
 '초등학교',
 '1학년생',
 '이',
 'ㄴ',
 '8살용영홬ㅋㅋ별반개',
 '도',
 '아까움',
 '원작',
 '의',
 '긴장감',
 '을',
 '제대로',
 '살',
 '려',
 '내',
 '지',
 '못하',
 '었다',
 '별',
 '반개',
 '도',
 '아깝',
 '다',
 '욕나온다',
 '이응경',
 '길용우',
 '연기생활이몇년인지정말',
 '발로해',
 '도',
 '그것보단',
 '낫겟다',
 '납치감금만반복반복이드라마',
 '는',
 '가족도',
 '없',
 '다',
 '연기못하는사람만모엿',
 '네',
 '액션',
 '이',
 '없',
 '는데',
 '도',
 '재미',
 '있',
 '는',
 '몇안되',
 '는',
 '영화',
 '왜케',
 '평점',
 '이',
 '낮',
 '은',
 '것',
 '이',
 'ㄴ데',
 '꽤',
 '보',
 'ㄹ',
 '만',
 '하',
 'ㄴ데',
 '헐리우드식',
 '화',
 '이',
 '리어',
 '하',
 'ㅁ',
 '에만',
 '너무',
 '길들이',
 '어',
 '지',
 '어',
 '있',
 '나',
 '걍인피니트가짱이다진짜짱',
 '이',
 '다',
 '보'

### - 6.2 nltk 객체 생성

In [17]:
text = nltk.Text(tokens)

### - 6.3 중복을 제외한 토큰의 수 

In [18]:
print(len(set(text.tokens)))

28713


### - 6.4 전체 문서에서 가장 많이 나온 토큰 10개 확인

In [19]:
print(text.vocab().most_common(10))

[('이', 12456), ('하', 5468), ('는', 4611), ('ㄴ', 4359), ('고', 3557), ('영화', 3261), ('가', 3182), ('어', 3047), ('의', 2978), ('보', 2950)]


## 7. Term-Document Matrix

In [20]:
vectorizer = CountVectorizer()
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

### - 7.1 train / test 데이터 나누기

In [21]:
train_docs = whole_data.reviews.values.tolist()[:len(train_data)]
test_docs= whole_data.reviews.values.tolist()[-len(test_data):]
train_docs

[['아', '더빙', '진짜', '짜증나', '네', '요', '목소리'],
 ['흠포스터보', '이', '고', '초딩영화줄오버연기조차', '가볍', '지', '않', '구나'],
 ['너무재밓었다그래서보는것을추천한다'],
 ['교도소', '이야기구먼', '솔직히', '재미', '는', '없다평점', '조정'],
 ['사이몬페그',
  '의',
  '익살',
  '스런',
  '연기',
  '가',
  '돋보이',
  '었던',
  '영화스파이더맨',
  '에서',
  '늙',
  '어',
  '보이',
  '기',
  '만',
  '하',
  '었던',
  '커스틴',
  '던스트',
  '가',
  '너무나',
  '도',
  '이쁘',
  '어',
  '보이',
  '었다'],
 ['막',
  '걸음마',
  '떼',
  'ㄴ',
  '3세',
  '부터',
  '초등학교',
  '1학년생',
  '이',
  'ㄴ',
  '8살용영홬ㅋㅋ별반개',
  '도',
  '아까움'],
 ['원작', '의', '긴장감', '을', '제대로', '살', '려', '내', '지', '못하', '었다'],
 ['별',
  '반개',
  '도',
  '아깝',
  '다',
  '욕나온다',
  '이응경',
  '길용우',
  '연기생활이몇년인지정말',
  '발로해',
  '도',
  '그것보단',
  '낫겟다',
  '납치감금만반복반복이드라마',
  '는',
  '가족도',
  '없',
  '다',
  '연기못하는사람만모엿',
  '네'],
 ['액션', '이', '없', '는데', '도', '재미', '있', '는', '몇안되', '는', '영화'],
 ['왜케',
  '평점',
  '이',
  '낮',
  '은',
  '것',
  '이',
  'ㄴ데',
  '꽤',
  '보',
  'ㄹ',
  '만',
  '하',
  'ㄴ데',
  '헐리우드식',
  '화',
  '이',
  '리어',
  '하',
  'ㅁ',
  '에만',
  '너무',
  '길들이',
  '어'

### - 7.2 train_docs 의 토큰 결합

In [22]:
train_docs = [' '.join(doc) for doc in train_docs]

In [23]:
train_docs[0]

'아 더빙 진짜 짜증나 네 요 목소리'

### - 7.3 test_docs 의 토큰 결합

In [24]:
test_docs = [' '.join(doc) for doc in test_docs]

In [25]:
test_docs[0]

'굳 ㅋ'

### - 7.4 Term-Document Matrix 생성 (train_x)

In [26]:
train_x = vectorizer.fit_transform(train_docs)

### - 7.5 Term-Document Matrix 생성 (test_x)

In [27]:
test_x = vectorizer.transform(test_docs)

### - 7.6 train_y와 test_y 생성

In [28]:
train_y = whole_data.sentiment.values.tolist()[:len(train_data)]
test_y = whole_data.sentiment.values.tolist()[-len(test_data):]

## 8. 분류 모델 생성 및 분류 성능 구하기

### - 8.1.1 SVM rbf 모델 생성 및 예측 

In [29]:
classifier_rbf = svm.SVC()
classifier_rbf.fit(train_x, train_y)
prediction_rbf = classifier_rbf.predict(test_x)

print(accuracy_score(test_y, prediction_rbf))

C:\Users\jiyoon\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.47285714285714286


### - 8.2.1 SVM linear 모델 생성 및 예측

In [30]:
classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(train_x, train_y)
prediction_linear = classifier_linear.predict(test_x)

print(accuracy_score(test_y, prediction_linear))

0.7271428571428571


### - 8.3.1 Random Forest 모델 생성 및 예측

In [31]:
rf = RandomForestClassifier()
rf.fit(train_x, train_y)
predict_rf = rf.predict(test_x)

print(accuracy_score(test_y, predict_rf))

C:\Users\jiyoon\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7571428571428571


### - 8.4.1 Gradient Boosting 분류 모델 생성 및 예측

In [32]:
gb = GradientBoostingClassifier()
gb.fit(train_x, train_y)
predict_gb = gb.predict(test_x)

print(accuracy_score(test_y, predict_gb))

0.6428571428571429


# Term-Document Matrix 

#### - SVM rbf                         : 0.48
#### - SVM linear                    : 0.72
#### - Random Forest            : 0.74
#### - Gradient Boosting       : 0.64


## 7. TF-IDF Matrix

In [33]:
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=5,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [34]:
train_docs = whole_data.reviews.values.tolist()[:len(train_data)]
test_docs  = whole_data.reviews.values.tolist()[-len(test_data):]

In [35]:
train_docs = [' '.join(doc) for doc in train_docs]
test_docs = [' '.join(doc) for doc in test_docs]

In [36]:
train_x = vectorizer.fit_transform(train_docs)
test_x = vectorizer.transform(test_docs)

In [37]:
classifier_rbf = svm.SVC()
classifier_rbf.fit(train_x, train_y)
prediction_rbf = classifier_rbf.predict(test_x)

print(accuracy_score(test_y, prediction_rbf))

C:\Users\jiyoon\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.47285714285714286


In [38]:
classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(train_x, train_y)
prediction_linear = classifier_linear.predict(test_x)

print(accuracy_score(test_y, prediction_linear))

0.7314285714285714


In [39]:
rf = RandomForestClassifier()
rf.fit(train_x, train_y)
predict_rf = rf.predict(test_x)

print(accuracy_score(test_y, predict_rf))

C:\Users\jiyoon\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6985714285714286


In [40]:
gb = GradientBoostingClassifier()
gb.fit(train_x, train_y)
predict_gb = gb.predict(test_x)

print(accuracy_score(test_y, predict_gb))

0.6385714285714286


# TF-IDF Matrix 

#### - SVM rbf                         : 0.47
#### - SVM linear                    : 0.73
#### - Random Forest            : 0.69
#### - Gradient Boosting       : 0.64